# Saver-like algorithm

In [1]:
# Import necessary modules
import Bio
from Bio import SeqIO
import csv


In [42]:
# load set1
set1_id=[]
set1_record=[]
for line in SeqIO.parse("Multiple_alignment.fa","fasta"):
    set1_id.append(line.id)
    set1_record.append(str(line.seq))

In [43]:
# load set2
set2_id=[]
set2_record=[]
for seq_record in SeqIO.parse("SaverPIKseq.fa","fasta"):
    set2_id.append((seq_record.id))
    set2_record.append(str(seq_record.seq))
len(set2_id)

372

In [44]:
set2exc1_id=[] #set2-set1 id
set2exc1_seq=[] #set2-set1 seq

for i in set2_id:
    if i not in set1_id:
        set2exc1_id.append(i)
        set2exc1_seq.append(set2_record[set2_id.index(i)])

In [71]:
clinvar=[]
with open('control.txt') as f:
    for line in f:
        clinvar.append(str(line))       
clean_clinvar=[]
for i in clinvar:
    new=i.strip("\n")
    clean_clinvar.append(new.split(" "))
clean_clinvar

[['NP_006209.2', '81', 'E', 'K'],
 ['NP_006209.2', '88', 'R', 'Q'],
 ['NP_006209.2', '91', 'D', 'N'],
 ['NP_006209.2', '93', 'R', 'W'],
 ['NP_006209.2', '93', 'R', 'Q'],
 ['NP_006209.2', '112', 'I', 'N'],
 ['NP_006209.2', '118', 'G', 'D'],
 ['NP_006209.2', '135', 'E', 'K'],
 ['NP_006209.2', '345', 'N', 'T'],
 ['NP_006209.2', '345', 'N', 'K'],
 ['NP_006209.2', '378', 'C', 'R'],
 ['NP_006209.2', '378', 'C', 'Y'],
 ['NP_006209.2', '420', 'C', 'R'],
 ['NP_006209.2', '545', 'E', 'V'],
 ['NP_006209.2', '545', 'E', 'S'],
 ['NP_006209.2', '545', 'E', 'A'],
 ['NP_006209.2', '909', 'F', 'L'],
 ['NP_006209.2', '914', 'G', 'R'],
 ['NP_006209.2', '1021', 'Y', 'C'],
 ['NP_006209.2', '1025', 'T', 'A'],
 ['NP_006209.2', '1035', 'A', 'V'],
 ['NP_006209.2', '134', 'P', 'S'],
 ['NP_006209.2', '83', 'F', 'S'],
 ['NP_006209.2', '115', 'R', 'P'],
 ['NP_006209.2', '183', 'N', 'S'],
 ['NP_006209.2', '218', 'E', 'K'],
 ['NP_006209.2', '299', 'M', 'V'],
 ['NP_006209.2', '344', 'V', 'M'],
 ['NP_006209.2', '356',

In [72]:
header=['Input', 'Prediction', 'Conservative', 'Substitutions_Set1', "N_Substitutions_Set1" 'Substitutions_Set2','N_Substitutions_Set2/Set1']

with open('saver_test.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    for x in clean_clinvar: 
        # Positions
        pos=int(x[1])-1 # position of interest in alligned sequences
        AA1=x[-1]

        # Human positions
        human=set1_record[0] # human sequence
        human_pos=set1_record[0][pos]

        # Gaps and X-amino acids
        gap="-"
        X="X"

        # Counters
        count=0 #counts how many amino acids identical to human are present in pos
        AA1_count=0 #counts how many AA1 are present in pos in set1
        AA1_set2excset1=0 #counts how many AA1 are present in pos in set2-set1
        X_count=0 #?
        gap_count=0 #?

        # Lists of amino acids
        sub_list=[] # will collect amino acids subsititions in set1
        set2_sub=[] # will collect amino acids subsititions in set2
        set2excset1_sub=[]

        # Human lists
        human_list=[human_pos,gap,X] 

        for seq in set1_record[1:]: # start from 1, because first(0) sequence is human

            if seq[pos] in human_list:
                count+=1
            elif seq[pos] == AA1:
                AA1_count+=1
                sub_list.append(seq[pos])
            else:
                sub_list.append(seq[pos])
        
        for seq2 in set1_record[1:]: # checking AA1 in set2
            
            if seq2[pos] == AA1:
                set2_sub.append(seq2[pos])
        
        for seq3 in set2exc1_seq: # counting AA1 in set2-set1
            
            if  seq3[pos] == AA1:
                AA1_set2excset1+=1
                set2excset1_sub.append(seq3[pos])
                
        
        # Editing the lists
        sub_list=list(set(sub_list))
        sub_list=(",".join(sub_list))
        sub_list=sub_list.replace(","," ")
        
        set2_sub=list(set(set2_sub))
        set2_sub=(",".join(set2_sub))
        set2_sub=set2_sub.replace(","," ")
        
        x=(",".join(x))
        x=x.replace(","," ")
                    
        if count == len (set1_record)-1: # AA0 fully conserved in set1 (first sequence is human)
            data=(x,'Damaging',count, sub_list, AA1_count, set2_sub, AA1_set2excset1)
            writer.writerow(data)
        elif AA1_count > 1: # Set1 has AA1 with > 1 events
            data=(x,'Benign',count, sub_list, AA1_count,set2_sub, AA1_set2excset1)
            writer.writerow(data)
        elif AA1_count ==1 and (AA1 in set2excset1_sub): #Set1 has single event of AA1 and AA1 is alse seen in set2-set1
            data=(x,'Benign',count, sub_list,AA1_count, set2_sub, AA1_set2excset1)
            writer.writerow(data)
        elif AA1 not in set2_sub: # AA1 not seen in set2
            data=(x,'Damaging',count, sub_list,AA1_count, set2_sub, AA1_set2excset1)
            writer.writerow(data)
        elif len(sub_list) < 5: # Position not moderately variable in set1 
            data=(x,'Damaging',count, sub_list,AA1_count, set2_sub, AA1_set2excset1)
            writer.writerow(data)
        elif len(sub_list) in range (5,10) and (AA1_set2excset1 >1): # Position moderately variable in set1 and set2-set1 has AA1 with >1 event
            data=(x,'Benign',count, sub_list,AA1_count, set2_sub, AA1_set2excset1)
            writer.writerow(data)
        elif len(sub_list) > 9: # Position hyper-variable in set 1
            data=(x,'Benign',count, sub_list,AA1_count, set2_sub, AA1_set2excset1)
            writer.writerow(data)
        else:
            data=(x,'Damaging',count, sub_list,AA1_count, set2_sub, AA1_set2excset1)
            writer.writerow(data)
            

In [73]:
import pandas as pd
predict = pd.read_csv('saver_test.csv')
predict.head(n=48)

,Input,Prediction,Conservative,Substitutions_Set1,N_Substitutions_Set1Substitutions_Set2,N_Substitutions_Set2/Set1
NP_006209.2 81 E K,Damaging,306,NaN,0,NaN,0
NP_006209.2 88 R Q,Damaging,306,NaN,0,NaN,0
NP_006209.2 91 D N,Damaging,306,NaN,0,NaN,0
NP_006209.2 93 R W,Damaging,306,NaN,0,NaN,0
NP_006209.2 93 R Q,Damaging,306,NaN,0,NaN,4
NP_006209.2 112 I N,Damaging,306,NaN,0,NaN,0
NP_006209.2 118 G D,Damaging,306,NaN,0,NaN,0
NP_006209.2 135 E K,Damaging,306,NaN,0,NaN,0
NP_006209.2 345 N T,Damaging,306,NaN,0,NaN,0
NP_006209.2 345 N K,Damaging,306,NaN,0,NaN,0


Discription of columns names
* Input - variants from clinvar or other database in NP_006209.2 1 M V format
* Prediction - prediction of the variant contribution to the disease
* Conservative - number of identical to human amino acids in the position of interest in set 1 
* Substitutions_Set1 - substitutions in the position of interest in set 1
* N_Substitutions_Set1 - number of substitutions in the position of interest in set 1
* Substitutions_Set2 - substitutions in the position of interest in set 2
* N_Substitutions_Set2/Set1 - number of substitutions in the position of interest in set2 -set1